In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import os

os.environ["BITSANDBYTES_NOWELCOME"] = "1"
if torch.cuda.is_available():
    print(f"VRAM used: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

print("Loading with proper quantization config...")

# Proper quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Load base model with proper config
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    quantization_config=quantization_config,
    torch_dtype=torch.float16,
    device_map="auto"
)



VRAM used: 4.04 GB
Loading with proper quantization config...


Loading checkpoint shards: 100%|██████████| 3/3 [00:15<00:00,  5.21s/it]


In [3]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load your trained adapter
model = PeftModel.from_pretrained(base_model, "./trained-models/paintings-audio-guide-final")
model.eval()

print("Model loaded successfully!")

Model loaded successfully!


In [4]:
# Test function
def test_model(title, artist, date, medium, dimensions="Not specified"):
    prompt = f"""Title: {title}
Artist: {artist}
Date: {date}
Medium: {medium}
Dimensions: {dimensions}

Audio guide:"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
        )
    
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    completion = result[len(prompt):].strip()
    
    # Clean up if it has <END>
    if "<END>" in completion:
        completion = completion.split("<END>")[0].strip()
    
    return completion

In [5]:
# Test 1: Girl with a Pearl Earring
print("=" * 60)
print("TEST 1: Girl with a Pearl Earring")
print("=" * 60)
result1 = test_model(
    "Girl with a Pearl Earring",
    "Johannes Vermeer",
    "c. 1665",
    "Oil on canvas",
    "17.5 × 15.6 in (44.5 × 39.4 cm)"
)
print(result1)

# Test 2: The Starry Night  
print("\n" + "=" * 60)
print("TEST 2: The Starry Night")
print("=" * 60)
result2 = test_model(
    "The Starry Night",
    "Vincent van Gogh",
    "1889",
    "Oil on canvas", 
    "29 × 36 1/4 in (73.7 × 92.1 cm)"
)
print(result2)

# Test 3: From your training data
print("\n" + "=" * 60)
print("TEST 3: Dutch Girl in White (From Training)")
print("=" * 60)
result3 = test_model(
    "Dutch Girl in White",
    "Robert Henri", 
    "1907",
    "Oil on canvas",
    "24 x 20 in. (61 x 50.8 cm)"
)
print(result3)

TEST 1: Girl with a Pearl Earring
Discover Girl with a Pearl Earring, where Johannes Vermeer's refined technique shines through this 17.5 × 15.6 in (44.5 × 39.4 cm) Oil on canvas. Created in c. 1665, the expressive lines demonstrates skill Notice the interplay of light and shadow reflecting the artistic values of its time

TEST 2: The Starry Night
Experience Vincent van Gogh's influential masterpiece The Starry Night, 29 × 36 1/4 in (73.7 × 92.1 cm) of Oil on canvas from 1889. The subtle approach The 29 × 36 1/4 in (73.7 × 92.1 cm) scale enhances the impact Vincent van Gogh's dynamic energy is evident throughout creating an

TEST 3: Dutch Girl in White (From Training)
Journey to the world of 1907 through Robert Henri's Dutch Girl in White, created in 1907. This 24 x 20 in. (61 x 50.8 cm) Oil on canvas showcases dynamic energy The dramatic lighting creates depth while the dynamic composition invites close contemplation
